## Contents

* [Setup](#Setup)
* [References](#References)

## Setup

In this notebook, we generate the experimental data required to carry out the statistical evaluation described in [1].
The data is logged into CSV files which will be later used when comparing the different MOCMA variants.

In [1]:
import os
import dataclasses
from itertools import product

import numpy as np
from IPython.display import Markdown

from anguilla.fitness import benchmark

from anguilla.evaluation import StopWatch, MOCMATrialParameters, LogParameters, log_mocma_trials

In [2]:
print(f"{os.cpu_count()} processors")

8 processors


In [3]:
# Search space dimension for both constrained and non-rotated benchmark functions is 30.

D_CONSTRAINED_NONROTATED = 30

# Search space dimension for rotated benchmark functions is 10.

D_ROTATED = 10

N_PARENTS = 100

N_TRIALS = 25

# The cartesian product of the following three global variables represents the space of 
# configured MOCMA-Benchmark function pairs for which we need to run N_TRIALS each

# We put the explicit expressions to compute the initial step size 
# for constrained functions as in p. 489 [1].
FUNCTIONS = [
    (benchmark.ZDT1, [D_CONSTRAINED_NONROTATED], 0.6 * (1.0 - 0.0)),
    (benchmark.ZDT2, [D_CONSTRAINED_NONROTATED], 0.6 * (1.0 - 0.0)),
    (benchmark.ZDT3, [D_CONSTRAINED_NONROTATED], 0.6 * (1.0 - 0.0)),
    (benchmark.ZDT4, [D_CONSTRAINED_NONROTATED], 0.6 * (1.0 - 0.0)),
    (benchmark.ZDT6, [D_CONSTRAINED_NONROTATED], 0.6 * (1.0 - 0.0)),
    (benchmark.IHR1, [D_ROTATED], 0.6 * (1.0 - -1.0)),
    (benchmark.IHR2, [D_ROTATED], 0.6 * (1.0 - -1.0)),
    (benchmark.IHR3, [D_ROTATED], 0.6 * (1.0 - -1.0)),
    (benchmark.IHR4, [D_ROTATED], 0.6 * (5.0 - -5.0)),
    (benchmark.IHR6, [D_ROTATED], 0.6 * (5.0 - -5.0)),
    (benchmark.ELLI1, [D_ROTATED], 1.0),
    (benchmark.ELLI2, [D_ROTATED], 1.0),
    (benchmark.CIGTAB1, [D_ROTATED], 1.0),
    (benchmark.CIGTAB2, [D_ROTATED], 1.0),
    (benchmark.DTLZ1, [D_CONSTRAINED_NONROTATED, 3], 0.6 * (1.0 - 0.0)),
    (benchmark.DTLZ2, [D_CONSTRAINED_NONROTATED, 3], 0.6 * (1.0 - 0.0)),
    (benchmark.DTLZ3, [D_CONSTRAINED_NONROTATED, 3], 0.6 * (1.0 - 0.0)),
    (benchmark.DTLZ4, [D_CONSTRAINED_NONROTATED, 3], 0.6 * (1.0 - 0.0)),
    (benchmark.DTLZ5, [D_CONSTRAINED_NONROTATED, 3], 0.6 * (1.0 - 0.0)),
    (benchmark.DTLZ6, [D_CONSTRAINED_NONROTATED, 3], 0.6 * (1.0 - 0.0)),
    (benchmark.DTLZ7, [D_CONSTRAINED_NONROTATED, 3], 0.6 * (1.0 - 0.0)),
    (benchmark.GELLI, [D_ROTATED, 3], 1.0),
]

# These next two global variables define the 4 MOCMA variants we will evaluate

OPTIMIZER_PARENT_OFFSPRING = [
    (100, 100),
    (100, 1),
]

OPTIMIZER_SUCCESS_NOTION = [
    "population",
    "individual",
]

# Generate the trial parameters for each job
TRIAL_PARAMETERS = []
for configuration in product(FUNCTIONS, OPTIMIZER_PARENT_OFFSPRING, OPTIMIZER_SUCCESS_NOTION):
    (fn_cls, fn_args, initial_step_size), (n_parents, n_offspring), success_notion = configuration
    TRIAL_PARAMETERS.append(
        MOCMATrialParameters(
            fn_cls,
            fn_args=fn_args,
            n_parents=n_parents,
            n_offspring=n_offspring,
            success_notion=success_notion,
            initial_step_size=initial_step_size,
        )
    )

# For reproducible results we set a base seed to create the seed 
# sequence used to generate children seeds for each independent trial
SEED = 90508458

# We want to checkpoint the best solutions at every 5000 iterations
LOG_PARAMETERS = LogParameters("./output/logs",
                               log_at=np.arange(5000, 55000, 5000, dtype=int).tolist(),
                               log_fitness=True,
                               log_points=False,
                               log_step_sizes=True,
                              )

In [ ]:
Markdown("""We run {} independent optimization trials for each of the {} optimizer-function pairs, which totals {} jobs.
            We sample 2 files every {} function evaluations (one file for the fitness and another one for the step sizes), 
            generating a total of {} CSV files.""".format(
    N_TRIALS,
    len(TRIAL_PARAMETERS),
    N_TRIALS * len(TRIAL_PARAMETERS),
    LOG_PARAMETERS.log_at[1] - LOG_PARAMETERS.log_at[0], 
    N_TRIALS * len(TRIAL_PARAMETERS) * len(LOG_PARAMETERS.log_at) * 2,
))

In [5]:
n = (N_TRIALS * len(TRIAL_PARAMETERS)) // 4

In [6]:
sw = StopWatch()
sw.start()
log_mocma_trials(dataclasses.replace(LOG_PARAMETERS, path="./output/logs-part1"),
           TRIAL_PARAMETERS,
           # If we partition with slices we can distribute the load between
           # multiple computers
           trial_slice=slice(0, n),
           seed=SEED,
           n_trials=N_TRIALS,
           n_processes=os.cpu_count(),
           chunksize=10,
          )
sw.stop()
print(f"Done! Took {sw.duration:.2f}s")

Number of jobs: 550

Number of processes: 8

Chunksize: 10

First job: MOCMATrialParameters(fn_cls=<class 'anguilla.fitness.benchmark.zdt.ZDT1'>, fn_args=[30], fn_kwargs={}, fn_rng_seed=None, n_parents=100, initial_step_size=0.6, success_notion='population', n_offspring=100, reference=None, region_bounds=None, max_generations=None, max_evaluations=None, target_indicator_value=None, key=1, seed=SeedSequence(
    entropy=90508458,
    spawn_key=(0,),
))

Last job: MOCMATrialParameters(fn_cls=<class 'anguilla.fitness.benchmark.ihr.IHR1'>, fn_args=[10], fn_kwargs={}, fn_rng_seed=None, n_parents=100, initial_step_size=1.2, success_notion='individual', n_offspring=100, reference=None, region_bounds=None, max_generations=None, max_evaluations=None, target_indicator_value=None, key=25, seed=SeedSequence(
    entropy=90508458,
    spawn_key=(549,),
))

Running 550 job(s) using parallel execution.
ZDT1-(100+100)-MO-CMA-ES-P-1
ZDT1-(100+100)-MO-CMA-ES-P-2
ZDT1-(100+100)-MO-CMA-ES-P-3
ZDT1-(100+10

In [7]:
sw = StopWatch()
sw.start()
log_mocma_trials(dataclasses.replace(LOG_PARAMETERS, path="./output/logs-part2"),
           TRIAL_PARAMETERS,
           trial_slice=slice(n, n*2),
           seed=SEED,
           n_trials=N_TRIALS,
           n_processes=os.cpu_count(),
           chunksize=10,
          )
sw.stop()
print(f"Done! Took {sw.duration:.2f}s")

Number of jobs: 550

Number of processes: 8

Chunksize: 10

First job: MOCMATrialParameters(fn_cls=<class 'anguilla.fitness.benchmark.ihr.IHR1'>, fn_args=[10], fn_kwargs={}, fn_rng_seed=None, n_parents=100, initial_step_size=1.2, success_notion='population', n_offspring=1, reference=None, region_bounds=None, max_generations=None, max_evaluations=None, target_indicator_value=None, key=1, seed=SeedSequence(
    entropy=90508458,
    spawn_key=(550,),
))

Last job: MOCMATrialParameters(fn_cls=<class 'anguilla.fitness.benchmark.elli.ELLI1'>, fn_args=[10], fn_kwargs={}, fn_rng_seed=None, n_parents=100, initial_step_size=1.0, success_notion='individual', n_offspring=1, reference=None, region_bounds=None, max_generations=None, max_evaluations=None, target_indicator_value=None, key=25, seed=SeedSequence(
    entropy=90508458,
    spawn_key=(1099,),
))

Running 550 job(s) using parallel execution.
IHR1-(100+1)-MO-CMA-ES-P-1
IHR1-(100+1)-MO-CMA-ES-P-2
IHR1-(100+1)-MO-CMA-ES-P-3
IHR1-(100+1)-MO-C

In [ ]:
sw = StopWatch()
sw.start()
log_mocma_trials(dataclasses.replace(LOG_PARAMETERS, path="./output/logs-part3"),
           TRIAL_PARAMETERS,
           trial_slice=slice(n*2, n*3),
           seed=SEED,
           n_trials=N_TRIALS,
           n_processes=os.cpu_count(),
           chunksize=10,
          )
sw.stop()
print(f"Done! Took {sw.duration:.2f}s")

In [ ]:
sw = StopWatch()
sw.start()
log_mocma_trials(dataclasses.replace(LOG_PARAMETERS, path="./output/logs-part4"),
           TRIAL_PARAMETERS,
           trial_slice=slice(n*3, n*4),
           seed=SEED,
           n_trials=N_TRIALS,
           n_processes=os.cpu_count(),
           chunksize=10,
          )
sw.stop()
print(f"Done! Took {sw.duration:.2f}s")

## References

[1] T. Voß, N. Hansen, and C. Igel. Improved Step Size Adaptation for the MO-CMA-ES. In Genetic And Evolutionary Computation Conference, 487–494. Portland, United States, July 2010. ACM. URL: https://hal.archives-ouvertes.fr/hal-00503251, doi:10.1145/1830483.1830573.

[2] C. Igel, N. Hansen, and S. Roth. Covariance matrix adaptation for multi-objective optimization. Evolutionary Computation, 15(1):1–28, 2007. doi:10.1162/evco.2007.15.1.1.